<a href="https://colab.research.google.com/github/ayushjain1144/semantic-segmentation-IGCAR/blob/master/bert_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/igcar_ps/ /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:

!git clone https://github.com/google-research/bert

Cloning into 'bert'...
remote: Enumerating objects: 329, done.
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (329/329), 239.74 KiB | 3.63 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [33]:

import os
import sys
import json
import random
import logging
import tensorflow as tf

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")



from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

auth.authenticate_user()

log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s : \
    %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    
    with tf.Session(TPU_ADDRESS) as session:
        log.info('TPU address is ' + TPU_ADDRESS)
        with open('/content/adc.json', 'r') as f:
          auth_info = json.load(f)
        
        tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
        
       
else:
    log.warning('Not connected to TPU runtime')
    USE_TPU = False

2019-07-06 06:18:40,507 :     Using TPU runtime
I0706 06:18:40.507112 139673349842816 interactiveshell.py:2882] Using TPU runtime
2019-07-06 06:18:40,513 :     TPU address is grpc://10.36.27.50:8470
I0706 06:18:40.513590 139673349842816 interactiveshell.py:2882] TPU address is grpc://10.36.27.50:8470


In [4]:
%cd /content/
!ls

/content
 adc.json  'combined_cleaned .exe'   gdrive
 bert	    final_data_newer.txt     sample_data


In [5]:
!ls

 adc.json  'combined_cleaned .exe'   gdrive
 bert	    final_data_newer.txt     sample_data


In [0]:

from bert import modeling, optimization, tokenization


In [0]:
import bert.run_pretraining
from bert.run_pretraining import input_fn_builder, model_fn_builder


In [0]:
MAX_SEQ_LENGTH = 128
MASKED_LM_PROB = 0.15
MAX_PREDICTIONS = 20
DO_LOWER_CASE = True
PROCESSES = 2
PRETRAINING_DIR = "pretraining_data"

In [11]:
!wc -w final_data_newer.txt

7618749 final_data_newer.txt


In [0]:
!mkdir ./shards
!split -a 4 -l 256000 -d 'final_data_newer.txt' ./shards/shard_

In [13]:
!ls ./shards/

shard_0000  shard_0001


In [14]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = '/mydrive/bert_uncased/' + BERT_MODEL
print('****** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR)) 
#!ls BERT_PRETRAINED_DIR

****** BERT pretrained directory: /mydrive/bert_uncased/uncased_L-12_H-768_A-12 *****


In [15]:
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt.*'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
!ls $CHKPT_DIR

/mydrive/bert_uncased/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
/mydrive/bert_uncased/uncased_L-12_H-768_A-12/bert_model.ckpt.index
/mydrive/bert_uncased/uncased_L-12_H-768_A-12/bert_model.ckpt.meta


In [16]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
            "python3 bert/create_pretraining_data.py "
            "--input_file=./shards/{} "
            "--output_file={}/{}.tfrecord "
            "--vocab_file={} "
            "--do_lower_case={} "
            "--max_predictions_per_seq={} "
            "--max_seq_length={} "
            "--masked_lm_prob={} "
            "--random_seed=108 "
            "--dupe_factors=5 ")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}',
                            PRETRAINING_DIR, '{}',
                            VOCAB_FILE,
                            DO_LOWER_CASE,
                            MAX_PREDICTIONS, MAX_SEQ_LENGTH,
                            MASKED_LM_PROB)

print(XARGS_CMD)

ls ./shards/ | xargs -n 1 -P 2 -I{} python3 bert/create_pretraining_data.py --input_file=./shards/{} --output_file=pretraining_data/{}.tfrecord --vocab_file=/mydrive/bert_uncased/uncased_L-12_H-768_A-12/vocab.txt --do_lower_case=True --max_predictions_per_seq=20 --max_seq_length=128 --masked_lm_prob=0.15 --random_seed=108 --dupe_factors=5 


In [0]:
tf.gfile.MkDir(PRETRAINING_DIR)


In [18]:
!$XARGS_CMD

W0706 05:42:11.592748 140503448741760 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:469: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0706 05:42:11.593875 140503448741760 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0706 05:42:11.594099 140503448741760 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0706 05:42:11.594341 140503448741760 deprecation_wrapper.py:119] From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0706 05:42:11.601556 140088198449024 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:469: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0706 05:42:11.602426 140088198449024 

In [0]:

BUCKET_NAME = "ayushjain1144-bucket"
MODEL_DIR = "bert_model"

if not BUCKET_NAME:
  log.warning("Warning: no bucket")



In [0]:
MODEL_DIR = "bert_model"
tf.gfile.MkDir(MODEL_DIR)

In [0]:

# hyperparameters for BERT BASE

bert_base_config = {
    "attention_probs_dropout_prob": 0.1,
    "directionality": "bidi",
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "max_position_embeddings": 512,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "pooler_fc_size": 768,
    "pooler_num_attention_heads": 12,
    "pooler_num_fc_layers": 3,
    "pooler_size_per_head": 128,
    "pooler_type": "first_token_transform",
    "vocab_size": 30522
}



In [22]:

!cp $BERT_CONFIG $MODEL_DIR/
!cp $CHKPT_DIR $MODEL_DIR/
!ls $MODEL_DIR/
    
with open("{}/bert_vocab.txt".format(MODEL_DIR), "w") as vocab:
    vocab_bert = open(VOCAB_FILE, 'r').read()
    vocab.write(vocab_bert)
    

bert_config.json		     bert_model.ckpt.index
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [31]:
!cp -r $MODEL_DIR $PRETRAINING_DIR /mydrive/

if BUCKET_NAME:
  !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME/original/
else:
  print("Not able to copy")

Copying file://bert_model/bert_config.json [Content-Type=application/json]...
Copying file://bert_model/bert_vocab.txt [Content-Type=text/plain]...
Copying file://pretraining_data/shard_0000.tfrecord [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://bert_model/bert_model.ckpt.meta [Content-Type=application/octet-st



```
# This is formatted as code
```



In [0]:
#TO DO
#code to transfer latest checkpoint from another directory to new directory



In [35]:
TRAIN_BATCH_SIZE = 128
MAX_PREDICTIONS =20
MAX_SEQ_LENGTH = 128
MASKED_LM_PROB = 0.15

EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 40000
SAVE_CHECKPOINTS_STEPS = 5000
NUM_TPU_CORES = 8

BERT_DRIVE_DIR = "{}/{}".format('/mydrive', MODEL_DIR)
DATA_DRIVE_DIR = "{}/{}".format('/mydrive', PRETRAINING_DIR)

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}/original".format(BUCKET_NAME)
else:
  print("bucket name not found")

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

PATH_TO_CHECKPOINT = os.path.join(BERT_GCS_DIR, "bert_model.ckpt")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)


if INIT_CHECKPOINT == None:
    print("no checkpoint found, loading the default")
    INIT_CHECKPOINT = PATH_TO_CHECKPOINT



CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")
VOCAB_FILE = os.path.join(BERT_GCS_DIR, "bert_vocab.txt")

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR, '*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

no checkpoint found, loading the default


2019-07-06 06:19:21,311 :     Using checkpoint: gs://ayushjain1144-bucket/original/bert_model/bert_model.ckpt
I0706 06:19:21.311156 139673349842816 interactiveshell.py:2882] Using checkpoint: gs://ayushjain1144-bucket/original/bert_model/bert_model.ckpt
2019-07-06 06:19:21,315 :     Using 2 data shards
I0706 06:19:21.315163 139673349842816 interactiveshell.py:2882] Using 2 data shards


In [36]:
model_fn = model_fn_builder(
    bert_config = bert_config,
    init_checkpoint= INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=TRAIN_STEPS,
    num_warmup_steps=4000,
    use_tpu=True,
    use_one_hot_embeddings=True)


tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
            cluster=tpu_cluster_resolver,
            model_dir=BERT_GCS_DIR,
            save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
            tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
            num_shards=NUM_TPU_CORES,
            per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

train_input_fn = input_fn_builder(
    input_files=input_files,
    max_seq_length=MAX_SEQ_LENGTH,
    max_predictions_per_seq=MAX_PREDICTIONS,
    is_training=True)


2019-07-06 06:19:35,883 :     Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f07f5b65950>) includes params argument, but params are not passed to Estimator.
W0706 06:19:35.883007 139673349842816 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f07f5b65950>) includes params argument, but params are not passed to Estimator.
2019-07-06 06:19:35,894 :     Using config: {'_model_dir': 'gs://ayushjain1144-bucket/original/bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.36.27.50:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_ex

In [0]:

estimator.train(input_fn=train_input_fn, max_steps = TRAIN_STEPS )


2019-07-06 06:19:44,140 :     Querying Tensorflow master (grpc://10.36.27.50:8470) for TPU system metadata.
I0706 06:19:44.140820 139673349842816 tpu_system_metadata.py:78] Querying Tensorflow master (grpc://10.36.27.50:8470) for TPU system metadata.
2019-07-06 06:19:44,160 :     Found TPU system:
I0706 06:19:44.160304 139673349842816 tpu_system_metadata.py:148] Found TPU system:
2019-07-06 06:19:44,164 :     *** Num TPU Cores: 8
I0706 06:19:44.164467 139673349842816 tpu_system_metadata.py:149] *** Num TPU Cores: 8
2019-07-06 06:19:44,169 :     *** Num TPU Workers: 1
I0706 06:19:44.169316 139673349842816 tpu_system_metadata.py:150] *** Num TPU Workers: 1
2019-07-06 06:19:44,172 :     *** Num TPU Cores Per Worker: 8
I0706 06:19:44.172520 139673349842816 tpu_system_metadata.py:152] *** Num TPU Cores Per Worker: 8
2019-07-06 06:19:44,175 :     *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12276970308501735927)
I0706 06:19:44.175703 13967334984

In [0]:
print('finished')